<a href="https://colab.research.google.com/github/Murcicrum/Identificador-de-medio/blob/main/identificador_de_medios_limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Imports

In [3]:
import os

import csv
import json

import requests
from bs4 import BeautifulSoup as bs
from time import sleep
from random import random

import re
import string

In [4]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Colab Notebooks/proyectos/identificador-de-medio/')

Mounted at /content/drive


In [5]:
!pip install stanza
import stanza
stanza.download('es')

nlp = stanza.Pipeline('es')

     |████████████████████████████████| 432 kB 5.0 MB/s 
     |████████████████████████████████| 175 kB 51.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=cf5ff8833fd54b18401e15c74aa49ffcbb22867863d997c923ea8887dc2d427e
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


2022-04-09 15:37:19 INFO: Downloading default packages for language: es (Spanish)...


2022-04-09 15:37:31 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-04-09 15:37:31 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-04-09 15:37:31 INFO: Use device: cpu
2022-04-09 15:37:31 INFO: Loading: tokenize
2022-04-09 15:37:31 INFO: Loading: mwt
2022-04-09 15:37:32 INFO: Loading: pos
2022-04-09 15:37:32 INFO: Loading: lemma
2022-04-09 15:37:32 INFO: Loading: depparse
2022-04-09 15:37:32 INFO: Loading: ner
2022-04-09 15:37:34 INFO: Done loading processors!


###Definiciones útiles

In [6]:
LETTERS = string.ascii_letters
SYMBOLS = string.punctuation + '¿¡'

#####Para leer los .json

In [7]:
def get_tweetstext(path_jsons)-> dict: 
    '''
    Dado el path a una carpeta que contenga archivos USER_tweets.json
    Devuelve diccionario con USER como claves 
    y una lista con los textos de sus tweets como valor
    '''
    tweets_text = {}
    
    for filename in os.listdir( path_jsons ):
      texts = []
      
      if filename.endswith('.json'):      
          fp = open(path_jsons+filename)      
          json_data = json.load( fp )
          fp.close()
      
      else: continue
      
      for element in json_data['data']:
          text = element['text']
          texts.append(text)

      user = filename.replace('_tweets.json', '')
      tweets_text[ user ] = texts
        
    return tweets_text



def print_tweets(tweets_dict, key=None):
    '''
    Dado un diccionario imprime ordenadamente su contenido
    Si una key del mismo es especificada, imprime sólo el valor de esa key
    Si no se especifica key se imprimen recursivamente todas 
    '''
    if key:
        print('***', key)
        for value in tweets_dict[key]:
           print('->', value)
    else:
        for key in tweets_dict:
            print_tweets(tweets_dict, key)
            print('\n\n')

#####Para scrapear Pagina12 e IP Noticias

Extraer los links a las publicaciones

In [8]:
def get_reallinks(twitter_links, verbose=False) -> list:
    '''
    Dada una lista de links comprimidos por twitter
    Devuelve los links reales/descomprimidos
    '''
    real_links = []
    
    for i, link in enumerate(twitter_links):
        if verbose: print('Procesando link ',i, '\t', link)
        
        try:
            response = requests.get(link)
            real_links.append(response.url)
        except: print('Falló lectura del link', i, '\t', link)
    
    return real_links


def get_links(tweets_list) -> list:
    links_list = []
    for tweet in tweets_list:
        i_link = tweet.rfind('http')                  #Capturo el último línk
         
        if i_link != -1:                              #i.e. si encontró un link, ya que si el tweet no tiene link rfind devuelve -1
            links_list.append( tweet[i_link:] )
        
    return links_list

Extraer los titulares de las publicaciones

In [9]:
def get_titles_ipn(ipn_links, verbose=False) -> list:
    '''
    Dada una lista de links de artículos de IPNoticias
    Devulve los titulares de dichos artículos
    '''
    ipn_titles = []

    for i,link in enumerate(ipn_links):
        if verbose: print('Link ', i, '\t',link)

#Me quedo con los links que remiten a notas escritas, descartando videos y directos
        if link.startswith('https://ip.digital/nota/'):
            response = requests.get(link)
            soup = bs(response.content)
            try:
                title = soup.find_all('title')[0].text
                i = title.find(' |')
                ipn_titles.append( title[:i] )
            except: print('Falló lectura del link ', i)
        
        elif verbose: print('El link no pertenece a una nota de IPNoticias.\n')
        
        sleep(random())
    return ipn_titles                


def get_titles_p12(p12_links, verbose=False) -> list:
    '''
    Dada una lista de links de artículos de Página12
    Devulve los titulares de dichos artículos
    '''
    p12_titles = []
    headers = {'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0'} #si no lo especificas el request devuelve HTTP-403
    
    for i, link in enumerate(p12_links):
        if verbose: print('Link ', i, '\t',link)
        
        if link.startswith('https://www.pagina12.com.ar/'):
          response = requests.get(link, headers=headers)
          soup = bs(response.content)
          
          try:
              title = soup.find_all('title')[0].text
              i = title.find(' |')
              p12_titles.append( title[:i] )
          except: print('Falló lectura del link ', i)
        
        elif verbose : print('El link no pertenece a una nota de Página12.\n')

        sleep(random())
    
    return p12_titles

#####Para lematizar los titulares

In [10]:
def lemmatize( string ):

    doc = nlp( string )

    lem_words = [ l for [l,u] in doc.get(['lemma','upos']) 
                    if u not in ['PUNCT','ADP','DET','PRON', 'CCONJ'] ]      #Filtro signos de puntuación, artículos, pronombres,etc. Definición de cada tag filtrado en: https://universaldependencies.org/u/pos/

    lem_string = ' '.join( lem_words )
    lem_string = lem_string.lower()         #Paso a minusculas al final para que los sustantivos propios no pierdan sentido durante la lematización. Ej: China -> chino

    return lem_string

#1. Cargado de datos

En nuestro caso los medios de los que extrajimos los tweets y sus respectivas cuentas fueron:

|  Medio | Cuenta de Twitter  |
|---|---|
|IP | _IPNoticias|
|Clarín | clarincom|
|Crónica | cronica|
|Infobae | Infobae|
|La Nación| LANACION|
|Página12| pagina12|
|TN| todonoticias|

###Lectura de archivos .json

Los tweets de los que partimos fueron obtenidos con la API de Twitter. Esta provee, para cada una de las cuentas de medios de noticias solicitadas,  un archivo .json de la forma:


```
{
    "data": [
        {
            "id": "1403823059149070336",
            "text": "\"Soy un ser superior\": c\u00f3mo fueron los macabros cr\u00edmenes del \"doctor muerte\", el m\u00e9dico que asesin\u00f3 a m\u00e1s de 200 pacientes https://t.co/zWmbnkvMyw"
        },
        {
            "id": "1403821608763113477",
            "text": "Fue al m\u00e9dico con el ojo irritado y descubri\u00f3 algo horroroso https://t.co/4uSkA1dCbW"
        },
        ...
        {
            "id": "1403821178200879107",
            "text": "RT @depoweb: \u00c1ngel Cabrera enfrentar\u00e1, en C\u00f3rdoba, el juicio por delitos de violencia de g\u00e9nero https://t.co/4dwEEPZtJg"
        }        
    ],
    "meta": {
        "newest_id": "1403824724422139905",
        "next_token": "7140dibdnow9c7btw3z0rtvf8v32vzyut25qapwcds7qv",
        "oldest_id": "1403818329819815937",
        "result_count": 10
    }
}
```



En primera instancia entonces lo que cabe es leer cada archivo .json y extraer de cada tweet ( i.e. de cada elemento de la lista con clave "data") el texto (string con clave "text"), para posteriormente aplicarlas las transformaciones necesarias para quedarnos sólo con los titulares limpios.

Extraigo entonces el texto de todos los archivos descargados:

In [11]:
tweets_text = get_tweetstext('tweets/')

Tenemos entonces, por ejemplo:

* Tweets de Crónica

In [12]:
print_tweets(tweets_text, key='cronica')

*** cronica
-> #VIDEO "No sabés con quién te estás metiendo": así presunto juez atacó a un motociclista en una escena a lo "Relatos Salvajes" #TremendaSecuencia #Tucumán #MiráLasImágenes https://t.co/yLwsUk0aM0
-> RT @BAENegocios: Restricciones en Santa Fe por coronavirus: fin de semana cerrado https://t.co/Jo02fXLv35
-> "No sabés con quién te estás metiendo": así presunto juez atacó a un motociclista en una escena a lo "Relatos Salvajes" https://t.co/Q0835OYfbW
-> "Soy un ser superior": cómo fueron los macabros crímenes del "doctor muerte", el médico que asesinó a más de 200 pacientes https://t.co/zWmbnkvMyw
-> Fue al médico con el ojo irritado y descubrió algo horroroso https://t.co/4uSkA1dCbW
-> RT @depoweb: Ángel Cabrera enfrentará, en Córdoba, el juicio por delitos de violencia de género https://t.co/4dwEEPZtJg
-> RT @depoweb: Goleador reveló que tanto Boca como River quisieron contratarlo https://t.co/PbsX9IY6gG
-> RT @depoweb: #Complicado El golfista ya está en su provincia y se

* Tweets de Clarín

In [13]:
print_tweets(tweets_text, key='clarincom')

*** clarincom
-> ¿Será cierto que el que apuesta a la inflación gana? https://t.co/6hDHxhGc5U
-> Joe Biden convence al G7 de lanzar un megaplan mundial de infraestructuras para hacer frente a China https://t.co/NcGehCmkKU
-> Hallazgo: el cine porno es un invento argentino como el dulce de leche https://t.co/UKKGk5TzgH
-> El capitán de Dinamarca, fundamental para salvarle la vida a Christian Eriksen https://t.co/AEhVCW5yoE
-> El intendente de Esteban Echeverría denunció que el Gobierno discrimina la entrega de ayuda alimentaria https://t.co/xprQXhsSdt
-> Indiana Jones cumple 40 años: la película que todos rechazaban y se convirtió en una de las sagas más exitosas del cine https://t.co/nkhTjKvp8c
-> Tras la polémica por el regreso de las clases presenciales, Axel Kicillof se reunió con Nicolás Trotta https://t.co/DG4agk6hbg
-> Coronavirus: reportan otras 447 muertes y Argentina superó los 85.000 fallecidos https://t.co/P2A6jG5qHr
-> Argentina superará las 20 millones de vacunas: llega es

* Tweets de Página12

In [14]:
print_tweets(tweets_text, key='pagina12')

*** pagina12
-> | La nueva ofensiva contra Milagro Sala de Gerardo Morales https://t.co/e0jQqeRVju
-> | Los investigadores creen que el padre drogó a ambas https://t.co/3TYaVNp7YF
-> Italia se estrenó este viernes en la Eurocopa con un contundente triunfo 3-0 contra Turquía, en el estadio Olímpico de Roma, por el Grupo A, del que forman parte también Suiza y a Gales. Los tantos fueron a... https://t.co/h9y7FqOE3G
-> Rusia probó una forma de aerosol nasal de su vacuna contra el coronavirus Sputnik V que es adecuada para niños de 8 a 12 años, y planea lanzar el nuevo producto en septiembre.Alexander Gintsburg, quien dirige el Instituto Gamaleya que des... https://t.co/KIrBZENLpC
-> | En un día sábado https://t.co/4Cbwc0Cmb1
-> | Lo capturó la Policía Federal https://t.co/VY0hY1f2jY
-> La Selección Argentina Sub 23 venció este viernes a Arabia Saudita por 2-0 en el segundo y último amistoso de la gira por Marbella, España, como preparación para los Juegos Olímpicos de Tokio.Tras un primer

###Scrapeo
En el caso de las cuentas de Página12 e IPNoticias, sus tweets no contienen el titular de la nota sino alguna frase que pertenece al copete o cuerpo de la misma. Para conseguir sus titulares lo que hago entonces es extraer los links que figuran en los tweets y scrapear las notas.

#####Extracción de links

Por lo general cada tweet finaliza con el link a la nota completa, pero en algunos casos este es precedido por otro link a algún sitio adicional, como puede verse en el caso de IP.

In [15]:
tweets_text['_IPNoticias']

['⚽ Confirmados los 28 para la #CopaAmerica2021 en #DEPORTIVOIP queremos saber qué pensás sobre el modo de Scaloni para definir la lista... 🔥👇',
 '💉 La Provincia de Buenos Aires llegó a 5.991.302 personas vacunadas contra el COVID-19, desde el comienzo de la pandemia. 👇\n\nhttps://t.co/1YJtcJ0Zle',
 '⚡️Los fanáticos de Marvel están felices: Loki, el Dios del Engaño, llega con serie propia en exclusiva para Disney Plus. Todos lo detalles te los cuentan @guilloleoz @yesroth y @rippyrizza en @MalditosNerdsVX 👇\n\nhttps://t.co/ctzcgKCce5',
 '🔴 La Ciudad de Buenos Aires registró un total de 2.015 nuevos contagios de COVID-19, en las últimas 24 horas. 👇\n\nhttps://t.co/TnsfnjmHlz',
 '💉🇦🇷 Argentina superó las 370 mil aplicaciones contra el COVID-19 en las últimas 24 horas, con un total de 372.518 en todo el territorio nacional. \n\n👉: https://t.co/rJmYAaMF16',
 '👉 Rodrigo Morabito, juez de Responsabilidad Juvenil N° 1 de la provincia de Catamarca, contó en #RedacciónIP la historia de un caso 

Pero estos links a su vez están comprimidos por twitter, no son explicitamente los links de los artículos; hay que ingresar en ellos para ser redirigidos a la nota en cuestión. Por ejemplo: https://t.co/e0jQqeRVju te redirige a https://www.pagina12.com.ar/347814-nueva-ofensiva-contra-milagro-sala-cronica-de-un-traslado-in?utm_term=Autofeed&utm_medium=Echobox&utm_source=Twitter#Echobox=1623530491. Con lo cual debo hacer un scraper intermedio que visite cada uno de los links comprimidos y extraiga de allí los links reales de los artículos.

In [16]:
links_p12 = get_links( tweets_text['pagina12'] )
links_p12 = get_reallinks( links_p12 )

links_ipn = get_links( tweets_text['_IPNoticias'] )
links_ipn = get_reallinks( links_ipn )

In [17]:
links_p12

['https://www.pagina12.com.ar/347814-nueva-ofensiva-contra-milagro-sala-cronica-de-un-traslado-in?utm_term=Autofeed&utm_medium=Echobox&utm_source=Twitter#Echobox=1623530491',
 'https://www.pagina12.com.ar/347808-horor-en-tenerife-se-dio-a-conocer-la-autopsia-de-la-nina-qu?utm_term=Autofeed&utm_medium=Echobox&utm_source=Twitter#Echobox=1623526894',
 'https://www.pagina12.com.ar/347674-italia-inauguro-la-eurocopa-2021-con-una-goleada-ante-turqui?utm_term=Autofeed&utm_medium=Echobox&utm_source=Twitter#Echobox=1623529667',
 'https://www.pagina12.com.ar/347813-coronavirus-rusia-prueba-la-vacuna-sputnik-v-en-aerosol-nasa?utm_term=Autofeed&utm_medium=Echobox&utm_source=Twitter#Echobox=1623529667',
 'https://www.pagina12.com.ar/347811-coronavirus-en-argentina-se-registraron-18-057-nuevos-casos-?utm_term=Autofeed&utm_medium=Echobox&utm_source=Twitter#Echobox=1623528635',
 'https://www.pagina12.com.ar/347803-el-represor-chileno-walter-klug-rivera-fue-detenido-en-caba?utm_term=Autofeed&utm_medium

In [18]:
links_ipn

['https://ip.digital/nota/6781-coronavirus-pba-se-cerca-6-millones-de-vacunados',
 'https://ip.digital/nota/6779-malditos-nerds-todo-sobre-loki-la-nueva-serie-de-marvel',
 'https://ip.digital/nota/6780-covid-19-caba-registro-2015-casos',
 'https://ip.digital/nota/6782-covid-19-argentina-supero-370-mil-vacunaciones',
 'https://ip.digital/nota/6772-la-victima-perdono-al-ladron-y-no-fue-la-carcel',
 'https://ip.digital/nota/6771-que-dice-el-dnu-que-comienza-regir-este-sabado',
 'https://twitter.com/_IPNoticias/status/1403531246668369920/video/1',
 'https://twitter.com/_IPNoticias/status/1403526461970124800/photo/1',
 'https://twitter.com/_IPNoticias/status/1403526269711572993/photo/1']

Más aún, los links de algunos tweets de IP no te dirigen a un artículo periodístico si no a fragmentos de video de sus programas emitidos por televisión.

#####Extracción de titulares
Ahora sí, teniendo los links a las publicaciones de ambos medios, puedo extraer sus titulares.

In [19]:
tweets_text['pagina12'] = get_titles_p12( links_p12 )

In [20]:
tweets_text['_IPNoticias'] = get_titles_ipn( links_ipn )

Obteniendo:

In [21]:
tweets_text['pagina12']

['Nueva ofensiva contra Milagro Sala: crónica de un "traslado intempestivo"',
 'Horror en Tenerife:\xa0se dio a conocer la autopsia de la niña que hallaron muerta en el mar',
 'Italia inauguró la Eurocopa 2021 con una goleada ante Turquía',
 'Coronavirus: Rusia prueba la vacuna Sputnik V en aerosol nasal',
 'Coronavirus en Argentina: se registraron 18.057 nuevos casos y 447 muertes',
 'El represor chileno Walther Klug Rivera fue detenido en CABA',
 'Los goles de la Selección Sub 23 frente a Arabia Saudita',
 'El lado luminoso de las canciones para seguir',
 'El fútbol estuvo en vilo por el episodio del danés Eriksen',
 'El tuit de Florencia Kirchner tras recibir el alta']

In [22]:
tweets_text['_IPNoticias']

['Coronavirus: PBA se cerca a 6 millones de vacunados',
 'Malditos Nerds: todo sobre Loki, la nueva serie de Marvel',
 'COVID-19: CABA registró 2.015 casos',
 'COVID-19: Argentina superó 370 mil vacunaciones',
 'La víctima perdonó al ladrón y no fue a la cárcel',
 '¿Qué dice el DNU que comienza a regir este sábado?']

Observesé que de esta forma los titulares ya salen limpios, sin caracteres especiales no alfanuméticos ni emojis, como si figuran en los tweets.

#2. Limpieza de los tweets

Lo siguientes es, para los tweets de todas las demas cuentas, eliminar de su contenido todo aquello que no sea el titular de la nota.

Recordemos que en todos los casos de todos los medios, el tweet es finalizado con un link, encontrarlo y eliminarlo es el primer paso en común para todos los medios.

In [23]:
def delete_links(tweets_list):
    tweets = tweets_list.copy()

    regex = re.compile( 'https.*' )
    tweets = [ re.sub( regex, '', t ) for t in tweets ]
    
    return tweets

Dado que cada medio adorna sus tweets de diferentes formas, hay una función para cada medio excepto para Página12 e IPNoticias que, como vimos, salen limpios del scrapeo.

Por lo general la "suciedad" en los tweets que queremos eliminar viene en forma de emojis, de hashtags no integrados en el texto, de menciones a los actores de la noticia, a sus redactores o a sus cuentas, o caracteres especiales utilizados para darle algún formato al tweet. Por ejemplo:

In [24]:
tweets_text['LANACION']

['Video: el ataque de furia de un hombre contra un motociclista tras un choque en Tucumán https://t.co/jhuPD8jvj4 https://t.co/fU2OXgMwdG',
 'Yao Cabrera vs el Chino Maidana: el youtuber mostró cómo se entrena para la esperada pelea https://t.co/khxA6h9CVU https://t.co/DGYKMv5Zs2',
 '“Matando al coronavirus”: la China Suárez mostró el tierno “protocolo” de su hija Magnolia https://t.co/MLSREmlMCN https://t.co/939SmcEQhx',
 'El drama de Delfina Gerez Bosco: tuvieron que operarla por un tumor de 750 gramos https://t.co/FH57dD9tbz https://t.co/gzfhzbIMMN',
 'Así resguardan las empresas su capital en pesos: la compra de oficinas premium y depósitos logísticos es una gran alternativa para convertir el capital en “ladrillos”. Brick Value, un plan para “dolarizar” el capital acumulado en pesos\n[#Contentlab para #Cushman&amp;Wakefield] https://t.co/kzlK2lShQP',
 'Copa América: entrenadores maduros y argentinos, el perfil de lo que se verá al borde de la cancha https://t.co/MD7RtgFZWJ https://

In [25]:
print(tweets_text['infobae'][5])

El Mercosur sigue paralizado: se suspendió la cumbre de cancilleres por las diferencias internas en el bloque | Por Eduardo Menegazzi
 https://t.co/S98mCB3ltr


Puede ser que incluso algunos medios publiquen un tweet que no referencie en absoluto a una nota periodística y sea sólo publicidad, una alerta de tránsito o clima, o un retweet a otra cuenta (no necesariamente informativa). 

In [26]:
print(tweets_text['cronica'][-1])

RT @depoweb: #Codiciado El jugador rechazó ambas ofertas porque prefirió seguir en Europa. Enterate de quién se trata, en la nota #Boca #Ri…


En estos casos casos optamos por descartar esos tweets.

Las funciones definidas a continuación sirven para limpiar el contenido de los tweets y obtener así los titulares limpios. 

NOTA: Estas funciones están escritas en el tiempo en que transcurrió el proyecto; en el fúturo las cuentas podrían cambiar los formatos de sus tweets y habría que adaptar estas funciones a esos cambios.

####TN

Mas o menos la mitad de los tweets finalizan bajando dos lineas y agregando el nombre de quien escribió la nota.

>El lenguaje inclusivo siempre hace ruido \\
 &nbsp; \\
Por Silvana Stabielli https://t.co/N2WlLfKHFo

>Adrián Barilari habló sobre su nuevo trabajo, su interés por el tango, y un clásico de Rata Blanca que compuso en el baño \\
&nbsp; \\
Por Carlos Iogna Prat https://t.co/2lSG1b7NNE



In [27]:
def clean_tn(tn_titles):
    titles = delete_links( tn_titles )
    
    titles  = [ t.split('\n\n')[0] for t in titles ]

    return titles

In [28]:
medio = 'todonoticias'
tweets_text[ medio ]  =  clean_tn( tweets_text[medio] )

#### La Nación

Casi todos los tweets ya tienen el titular limpio, pero finalizan con dos links: uno a la nota completa y otro al mismo tweet.
Hay algunos pocos que finalizan con alguna mención a entidades que participaron en la nota, como por ejemplo:



>  'Así resguardan las empresas su capital en pesos: la compra de oficinas premium y depósitos logísticos es una gran alternativa para convertir el capital en “ladrillos”. Brick Value, un plan para “dolarizar” el capital acumulado en pesos \\
[#Contentlab para #Cushman&amp;Wakefield] https://t.co/kzlK2lShQP'




In [29]:
def clean_ln(ln_titles):
    titles = delete_links( ln_titles )
    titles = delete_links( titles )

    regex = re.compile( '\\n\[.*\]' )
    titles = [ re.sub(regex, '',t) for t in titles ]
    return titles

In [30]:
medio = 'LANACION'
tweets_text[medio] = clean_ln( tweets_text[medio] )

####Clarín

En la gran mayoría de los tweets de Clarín tienen el titular limpio.
Sin embargo hay algunos pocos que inician con un emoji y terminan dando pié a la nota con un "Más: "
> 👉El director del Instituto Gamaleya señaló que las dos dosis también son efectivas contra la variante Delta, la más contagiosa del mundo. Más: https://t.co/3FrwtVAKxH https://t.co/KYlfKknrth


In [31]:
def clean_clarin(clarin_titles):    
    titles = delete_links( clarin_titles )
    
    #Descarto los que inician con emoji (que no es letra ni simbolo)
    titles = [ title[1:] if title[0] not in (LETTERS+SYMBOLS) 
                         else title
                         for title in titles] 
                         
    #Elimino el "Más:" posterior al titular, que daba pié al link con la nota completa
    titles = [ t.replace('. Más:', '') for t in titles ]
        
    return titles

In [32]:
medio = 'clarincom'
tweets_text[ medio ]  =  clean_clarin( tweets_text[medio] )

####Crónica

Los titulares a limpiar suelen ser de las siguientes formas.


Tienen hashtags no integrados al titular
> '#VIDEO "No sabés con quién te estás metiendo": así presunto juez atacó a un motociclista en una escena a lo "Relatos Salvajes" #TremendaSecuencia #Tucumán #MiráLasImágenes https://t.co/yLwsUk0aM0'

Son retweets de otros medios o usuarios
> 'RT @depoweb: Goleador reveló que tanto Boca como River quisieron contratarlo https://t.co/PbsX9IY6gG'

Inician con una palabra o frase entre emojis e incluyen el subtitulo de la nota luego de caracteres ">"
> ❤️[EMOTIVO]❤️ El conmovedor tatuaje de Ricky Sarkany en honor a su hija Sofía: "Para siempre" &gt;&gt;&gt; El diseñador compartió el dibujo con el que inmortalizó en su piel a su primogénita quien falleció en marzo tras una dura batalla contra el cáncer de útero. https://t.co/iKqCrbJbPq


In [33]:
def clean_cronica(cronica_titles):
    titles = delete_links(cronica_titles)

    titles = [ t for t in titles if 'RT @' not in t ]

    titles = [ t.split('&gt;')[0] for t in titles ]
    
    regex = re.compile( '.\[.*\].' )
    titles = [ re.sub(regex, '',t) for t in titles ]

    regex = re.compile( '#\S+' )
    titles = [ re.sub(regex,'',t) for t in titles ]

    return titles

In [34]:
tweets_text['cronica'] = clean_cronica(tweets_text['cronica'])
tweets_text['cronica']

[' "No sabés con quién te estás metiendo": así presunto juez atacó a un motociclista en una escena a lo "Relatos Salvajes"    ',
 '"No sabés con quién te estás metiendo": así presunto juez atacó a un motociclista en una escena a lo "Relatos Salvajes" ',
 '"Soy un ser superior": cómo fueron los macabros crímenes del "doctor muerte", el médico que asesinó a más de 200 pacientes ',
 'Fue al médico con el ojo irritado y descubrió algo horroroso ']

Observese que los primeros dos titulares que sobrevivieron a la limpieza son idénticos, con lo cual también vamos a tener que eliminar duplicados.

#### Infobae

Algunos finalizan con el nombre del autor del artículo escrito como  "| Por *Nombre* *Apellido*"

>Dos gigantes multinacionales ofrecen más de 2.000 puestos de trabajo en la Argentina: qué perfiles buscan | Por Ximena Casas
 https://t.co/NBOHzQxaql

Pueden iniciar también con una palabra o frase en mayúsculas entre corchetes:

>[OPINIÓN] Hacer seguidismo con México no sustituye la falta de política con Brasil y el Mercosur | Por Mariano Caucino
 https://t.co/3GACDT6Lj5

Iniciar con emojis y palabras en mayuscula
>🔴 AHORA | La ciudad de Buenos Aires abrirá la inscripción para que se vacunen los mayores de 40 años sin comorbilidades https://t.co/1mJIxMB2qJ

O bien tener un doble link al final o ser retweets.

In [35]:
def clean_infobae(infobae_titles):
    titles = delete_links( infobae_titles )
    titles = delete_links( titles )

    titles = [ t for t in titles if 'RT @' not in t ]

    regex = re.compile( '. ([A-Z]\S+) \| ' )
    titles = [re.sub(regex, '', t) for t in titles ]

    titles = [ t.split('| Por')[0] for t in titles ]

    return titles

In [36]:
medio = 'infobae'
tweets_text[medio] = clean_infobae( tweets_text[medio] )

####Observación importante

Originalmente habíamos partido con 10 tweets de cada cuenta. Ahora, luego de a limpieza nos quedaron para cada medio:

In [37]:
print('CUENTA \t\t Nº DE TWEETS\n')
for k in tweets_text.keys():
    print(k,'\t\t', len(tweets_text[k]))

CUENTA 		 Nº DE TWEETS

clarincom 		 10
cronica 		 4
infobae 		 10
_IPNoticias 		 6
LANACION 		 10
pagina12 		 10
todonoticias 		 10


Tenemos entonces ya un desbalance de clases causado por la limpieza de los datos. Y recordemos también que podría haber titulares duplicados, como en el caso de crónica.

Por último cabe decir que con estas funciones podemos recuperar de los tweet la inmensa mayoría de los titulares. Pero de todas formas es bueno inspeccionar el resultado final, constatar que las listas contengan los titulares limpios como corresponde, y limpiar a mano algún tweet particular que haya tenido un formato específico que no haya sido capturado por las funciones.

#3. Armado de dataset

Quiero transformar ```tweets_text``` en una lista de diccionarios de la forma:


```
data = [ {'medio':'m1', 'titular':'t1', 'titular_lematizado':'n1'},
         {'medio':'m1', 'titular':'t2', 'titular_lematizado':'n2'},
         {'medio':'m2', 'titular':'t3', 'titular_lematizado':'n3'},
         {'medio':'m3', 'titular':'t4', 'titular_lematizado':'n4'},
         {'medio':'m3', 'titular':'t5', 'titular_lematizado':'n5'},
         {'medio':'m3', 'titular':'t6', 'titular_lematizado':'n6'},
         ...]
```
Para poder luego guardarlo luego en un csv limpio más facilmente.

Necesito primero definir una función que, dado un titular, me lo devuelva lematizado y, de paso, sin signos de puntuación. Para lograr esto probamos varias librerias de NLP y la que nos dió mejores resultados fué Stanza.

In [51]:
data = []

for medio in tweets_text.keys():
    for titular in tweets_text[medio]:
        d = {'medio':medio,
             'titular_raw':titular,
             'titular_lem': lemmatize( titular ) }

        data.append(d)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


Y así conseguimos los titulareslematizados con los que alimentamos el modelo:

In [50]:
data[20:30]

[{'medio': 'infobae',
  'titular_lem': 'calefaccionar balcón terraza opción acondicionar espacio aire libre nuevo normalidad invierno',
  'titular_raw': 'Cómo calefaccionar el balcón y la terraza: opciones para acondicionar los espacios al aire libre a la nueva normalidad en invierno '},
 {'medio': 'infobae',
  'titular_lem': 'gobierno reducir vuelo europa suspender nuevo destino evitar ingreso más cepa covid-19',
  'titular_raw': 'El Gobierno redujo los vuelos con Europa y suspendió nuevos destinos para evitar el ingreso de más cepas del COVID-19 '},
 {'medio': 'infobae',
  'titular_lem': '77 foto primero dos semana showmatch backstage truco participante transformación humorista',
  'titular_raw': '77 fotos de las primeras dos semanas de ShowMatch: el backstage, los trucos de los participantes y la transformación de los humoristas '},
 {'medio': 'infobae',
  'titular_lem': 'amar christian grito gol romelu lukaku dedicado eriksen recorrer mundo',
  'titular_raw': '“¡Te amo, Christian!”


#4. Guardado

Finalmente gurdo esto en un archivo .csv para poder poder leerlo en la siguient etapa.

In [52]:
FILENAME = 'titulares_muestra.csv'

with open(FILENAME, 'w') as file:

    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    
    writer.writeheader()
    writer.writerows( data )